##### 1) 라이브러리 설치

In [ ]:
# poetry add faiss-cpu

##### 2) OpenAI 인증키 설정


In [ ]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:3])

##### Vector Store Retriever
* 벡터 저장소(Vector Store)를 활용하여 사용자의 쿼리에 대한 가장 유사한 문서를 검색

In [ ]:
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_ollama import OllamaEmbeddings
from langchain.schema import Document

# 문서 리스트 생성
documents = [
    Document(page_content="LangChain은 LLM 기반 애플리케이션을 쉽게 구축할 수 있도록 지원합니다."),
    Document(page_content="벡터 저장소는 문서를 벡터화하여 빠른 검색을 가능하게 합니다."),
    Document(page_content="멀티 쿼리 검색은 한 가지 질문을 여러 방식으로 변형하여 검색 효율을 높입니다."),
    Document(page_content="LangChain은 해리슨 체이스(Harrison Chase)가 2022년 10월에 개발되었습니다."),
    Document(page_content="LangChain의 주요 기능 중 하나는 다양한 LLM 모델과의 연동입니다."),
    Document(page_content="LangChain Core 라이브러리는 기본 추상화 및 LangChain 표현 언어"),
    Document(page_content="LangChain Community 라이브러리는 서드파티 통합"),
]

# 임베딩 모델 설정
#embedding_model = OpenAIEmbeddings()
embedding_model = OllamaEmbeddings(model="bge-m3:latest")

# FAISS 기반 벡터 저장소 생성
vector_store = FAISS.from_documents(documents, embedding_model)

# search_kwargs의 기본값은 {"k": 3} 기본적으로 최대 3개의 문서를 검색
# 필요에 따라 k 값을 조정하여 검색 문서 개수를 변경할 수 있음
# search_kwargs={"k": 4}
#retriever = vector_store.as_retriever()
#print(type(retriever))

filtered_retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.3, "k": 4} # 임계값을 0.3으로 낮춤
)

# 검색 실행
query = "LangChain 이란?"
retrieved_docs = filtered_retriever.invoke(query)

# 검색된 문서 출력
for doc in retrieved_docs:
    print(doc.page_content)

##### Multi Query Retriever

In [ ]:
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_ollama import OllamaEmbeddings
from langchain_openai import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever

# 문서 리스트 생성
documents = [
    Document(page_content="LangChain은 LLM 기반 애플리케이션을 쉽게 구축할 수 있도록 지원합니다."),
    Document(page_content="벡터 저장소는 문서를 벡터화하여 빠른 검색을 가능하게 합니다."),
    Document(page_content="멀티 쿼리 검색은 한 가지 질문을 여러 방식으로 변형하여 검색 효율을 높입니다."),
    Document(page_content="LangChain은 해리슨 체이스(Harrison Chase)가 2022년 10월에 개발되었습니다."),
    Document(page_content="LangChain의 주요 기능 중 하나는 다양한 LLM 모델과의 연동입니다."),
    Document(page_content="LangChain Core 라이브러리는 기본 추상화 및 LangChain 표현 언어"),
    Document(page_content="LangChain Community 라이브러리는 서드파티 통합"),
]

# 임베딩 및 벡터스토어 설정
#embedding_model = OpenAIEmbeddings()
embedding_model = OllamaEmbeddings(model="bge-m3:latest")
vector_store = FAISS.from_documents(documents, embedding_model)

# LLM 모델 설정 (GPT 사용)
llm = ChatOpenAI(model_name="gpt-4o-mini")
# llm = ChatOpenAI(
#     base_url="https://api.groq.com/openai/v1",
#     model="moonshotai/kimi-k2-instruct-0905",
#     temperature=0
# )

# Multi Query Retriever 생성
retriever = MultiQueryRetriever.from_llm(retriever=filtered_retriever, llm=llm)

# 검색 실행
query = "LangChain 이란?"
#query = "오늘의 날씨는?"
retrieved_docs = retriever.invoke(query)
print(len(retrieved_docs))

# 검색된 문서 출력
for doc in retrieved_docs:
    print(doc.page_content)

##### VectorStoreRetriever 와 MultiQueryRetriever 비교하기

In [ ]:
from langchain.vectorstores import FAISS
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_ollama import OllamaEmbeddings
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.schema import Document
import logging

# 로깅 설정 (쿼리 변형 과정 확인)
logging.basicConfig(level=logging.INFO)

# 문서 리스트 생성
documents = [
    Document(page_content="LangChain은 LLM 기반 애플리케이션을 쉽게 구축할 수 있도록 지원합니다."),
    Document(page_content="벡터 저장소는 문서를 벡터화하여 빠른 검색을 가능하게 합니다."),
    Document(page_content="멀티 쿼리 검색은 한 가지 질문을 여러 방식으로 변형하여 검색 효율을 높입니다."),
    Document(page_content="LangChain은 해리슨 체이스(Harrison Chase)가 2022년 10월에 개발되었습니다."),
    Document(page_content="LangChain의 주요 기능 중 하나는 다양한 LLM 모델과의 연동입니다."),
    Document(page_content="LangChain Core 라이브러리는 기본 추상화 및 LangChain 표현 언어"),
    Document(page_content="LangChain Community 라이브러리는 서드파티 통합"),
]

# 임베딩 모델 및 벡터스토어 설정
#embedding_model = OpenAIEmbeddings()
embedding_model = OllamaEmbeddings(model="bge-m3:latest")
vector_store = FAISS.from_documents(documents, embedding_model)

# LLM 모델 설정 (GPT 사용)
llm = ChatOpenAI(model_name="gpt-4o-mini")
# llm = ChatOpenAI(
#     base_url="https://api.groq.com/openai/v1",
#     model="moonshotai/kimi-k2-instruct-0905",
#     temperature=0
# )

# 일반 VectorStore Retriever (기본 검색)
vector_retriever = vector_store.as_retriever()

# Multi Query Retriever 설정
multi_query_retriever = MultiQueryRetriever.from_llm(retriever=vector_retriever, llm=llm)
#multi_query_retriever = MultiQueryRetriever.from_llm(retriever=vector_store.as_retriever(search_kwargs={"k": 4}), llm=llm)

# 검색 실행 (LangChain 관련 질문)
query = "LangChain 이란?"

print("\n [1] 일반 VectorStore Retriever 검색 결과:")
retrieved_docs_vr = vector_retriever.invoke(query)
for doc in retrieved_docs_vr:
    print(f" - {doc.page_content}")

print("\n [2] Multi Query Retriever가 생성한 다양한 검색 쿼리:")
multi_query_retriever._log = True  # 변형된 쿼리 로깅 활성화
retrieved_docs_mqr = multi_query_retriever.invoke(query)

print("\n [3] Multi Query Retriever 검색 결과:")
for doc in retrieved_docs_mqr:
    print(f" - {doc.page_content}")